---

# App

---

for accessing an additional [default container](https://forum.safedev.org/t/safe-network-api-getting-started-draft/726) you need to add container-permissions and count up the container length

```python
import safenet
safenet.setup_logger(file_level=safenet.log_util.DEBUG)
myApp = safenet.App()
newPermissions,permission_addon=safenet.safe_utils.ContainerPermissions(b'_music',ffi=myApp.ffi_app)
myAuth_,addData=safenet.safe_utils.AuthReq(newPermissions,1,1,id=b'newProgId_unique_thing',scope=b'lalali'
                       ,name=b'newProgramName',vendor=b'nobody',app_container=True,ffi=myApp.ffi_app)
```

in this simple case we just want access to the default container and set the logger to a medium level:

In [1]:
import safenet
safenet.setup_logger(file_level=safenet.log_util.DEBUG)
myApp = safenet.App()
myAuth_,addData=safenet.safe_utils.AuthReq(myApp.ffi_app.NULL,0,0,id=b'otherProgramname',scope=b'noScope'
                       ,name=b'oneProg',vendor=b'no_vendor',app_container=True,ffi=myApp.ffi_app)

SAFE python-rust interface generated, bound and available
[19:08:59.227] pyS        :MainThread(       log_util.py: 112) [   DEBUG] logger initialized. File: True, Stream: True                    
[19:08:59.503] pyS.ffi_in :Thread-5  (     safe_utils.py: 137) [    INFO] action succeeded: < _auth_exe_file_stem_o_cb                    
[19:08:59.504] pyS.ffi_in :Thread-4  (     safe_utils.py: 137) [    INFO] action succeeded: < _auth_set_additional_search_path_o_cb       


In [2]:
myApp.encode_auth_req(myAuth_,myApp.ffi_app.NULL)
encodedAuth = myApp.queue.get()

[19:08:59.904] pyS.ffi_in :Thread-6  (     safe_utils.py: 137) [    INFO] action succeeded: < _encode_auth_req_o_cb                       


In [3]:
grantedAuth = myApp.sysUri.quickSetup(myAuth_,encodedAuth)

[19:09:00.634] pyS.ffi_out:Thread-7  (safe_sysUri_defs.py:  30) [   DEBUG] sysUri install called                                           
[19:09:00.634] pyS        :MainThread(         sysUri.py:  63) [    INFO] filename is: b'/tmp/tmpx48v1zkn' and port number is: 7035       
[19:09:00.669] pyS.ffi_out:Thread-8  (safe_sysUri_defs.py:  54) [   DEBUG] sysUri open_uri called                                          
[19:09:00.683] pyS.ffi_in :Thread-7  (     safe_utils.py: 137) [    INFO] action succeeded: < _install_o_cb                               
[19:09:02.096] pyS.ffi_in :Thread-8  (     safe_utils.py: 137) [    INFO] action succeeded: < _open_uri_o_cb                              


the granted auth (a long string) then needs to be decoded to get the authGranted-pointer from the safe-api

In [4]:
myApp.decode_ipc_msg(grantedAuth,None)

In [5]:
grantedAuthPointer = myApp.queue.get()

that is then used to get the app-pointer

In [6]:
myApp.app_registered(myAuth_.app.id,grantedAuthPointer[0],None)

In [7]:
appPointer = myApp.queue.get()

[19:09:11.254] pyS.ffi_in :Thread-10 (     safe_utils.py: 137) [    INFO] action succeeded: < _app_registered_o_cb                        


---

### now we have an app and can start doing stuff

---

In [8]:
with open('myNewMutableData_as_bytes','rb') as f:
    mutableBytes = f.read()

In [9]:
ffiMut = safenet.safe_utils.getffiMutable(mutableBytes,myApp.ffi_app)

In [42]:
@myApp.ffi_app.callback("void(void* , FfiResult*, MDataKey*, uint64_t)")
def result_mdata_list_keys(user_data, result, key, size):
    #print('results')
    #print('error code: ')
    #print(result.error_code)
    if result.error_code != 0:
        print(myApp.ffi_app.string(result.description))
    #print(key.val_len)
    #print(size)
    if size>0:
        #print(key.val_len)
        print(myApp.ffi_app.string(key.key))
    #returnDict = ffi.from_handle(user_data)
    #returnDict['myMdata']=mdataInfo

In [43]:
@myApp.ffi_app.callback("void(void* , FfiResult*, MDataValue*, uint64_t)")
def result_mdata_list_values(user_data, result, value, size):
    #print('results')
    #print('error code: ')
    #print(result.error_code)
    if result.error_code != 0:
        print(myApp.ffi_app.string(result.description))
    #print(key.val_len)
    #print(size)
    if size>0:
        #print(value.content_len)
        print(myApp.ffi_app.string(value.content))
    #returnDict = ffi.from_handle(user_data)
    #returnDict['myMdata']=mdataInfo

In [44]:
myApp.lib.safe_app.mdata_list_keys(appPointer,ffiMut,myApp.ffi_app.NULL,result_mdata_list_keys)

b'dask'


In [45]:
myApp.lib.safe_app.mdata_list_values(appPointer,ffiMut,myApp.ffi_app.NULL,result_mdata_list_values)

b'we get closer...'


In [10]:
myMutable = safenet.MutableData(mutableBytes)

In [11]:
myMutable.mdata_list_values(appPointer,ffiMut,None)

[19:09:21.767] pyS.ffi_out:Thread-11 (  safe_app_defs.py: 423) [   DEBUG] checking if the ffi exists                                      
[19:09:21.768] pyS.ffi_out:Thread-11 (  safe_app_defs.py: 424) [   DEBUG] <cffi.api.FFI object at 0x7fd75825efd0>                         
[19:09:21.769] pyS.ffi_out:Thread-11 (  safe_app_defs.py: 425) [   DEBUG] <bound method FFI.callback of <cffi.api.FFI object at 0x7fd75825efd0>>
[19:09:21.771] pyS.ffi_out:Thread-11 (  safe_app_defs.py: 426) [   DEBUG] <cffi.api._make_ffi_library.<locals>.FFILibrary object at 0x7fd74aebd6d8>
[19:09:21.773] pyS.ffi_out:Thread-11 (  safe_app_defs.py: 427) [   DEBUG] <cdata 'void(*)(void * *, MDataInfo *, void *, void(*)(void *, FfiResult *, MDataValue *, unsigned long))' 0x7fd74a0edda0>
[19:09:21.774] pyS.ffi_out:Thread-11 (  safe_app_defs.py: 428) [   DEBUG] attempting to return mdataValues                                
[19:09:21.775] pyS.ffi_out:Thread-11 (  safe_app_defs.py: 429) [   DEBUG] app has type <cdata

In [ ]:
myMutable.mdata_permissions_new(appPointer,None)

[16:17:17.934] pyS.ffi_out:Thread-13 (  safe_app_defs.py: 557) [   DEBUG] attempting to get a permission handle with userdata <cdata 'void *' NULL> and app <cdata 'void * *' 0x7f8fa0049d60>


In [14]:
permissionHandle = myMutable.queue.get_nowait()

In [15]:
permissionHandle

1

In [13]:
myMutable.mdata_entries_new(appPointer,None)

[16:17:07.237] pyS.ffi_out:Thread-11 (  safe_app_defs.py: 814) [   DEBUG] attempting to get a entries handle with userdata <cdata 'void *' NULL> and app <cdata 'void * *' 0x7f8fa0049d60>
[16:17:07.240] pyS.ffi_in :Dummy-12  (     safe_utils.py: 137) [    INFO] action succeeded: < _mdata_entries_new_o_cb                     


In [14]:
entryHandle = myMutable.queue.get_nowait()

In [15]:
entryHandle

1

In [14]:
myMutable.test(appPointer,None)

AttributeError: test

In [24]:
myApp.mutableData.(mdata_permissions_new(appPointer,None))

From cffi callback <function OutStream.write.<locals>.<lambda> at 0x7f85bcee2510>:
TypeError: _mdata_permissions_new_o_cb() missing 3 required positional arguments: 'user_data', 'result', and 'perm_h'


In [1]:
permissionHandle

NameError: name 'permissionHandle' is not defined

In [22]:
appPointer

<cdata 'void * *' 0x7f85b80c8c90>

In [4]:
grantedAuth

'bAEAAAAEMVWZR2AAAAAAAAAAAAAQAAAAAAAAAAACGXDI7FJQJGFWTNW3C6KKWWNKBOOU4OVE3T23X7XQZHHMYK6SFOMQAAAAAAAAAAAFGOIG3WJ3KMXRCCSNVNM3OYESI5S5VJZJHUW2AKWHM24FUXZAUMQQAAAAAAAAAAAFMS46SBKZB7SAXDL7TK6RHYR5MV6PUEO3IULWJPPTGX4WC75CTUFAAAAAAAAAAAAEPIUOBVE2QSQTAG3FRNKOW7CWMHWQAGJNO45CKT4B7WJUFUARH6CWJOPJAVMQ7ZALRV7ZVPIT4I6WK7H2CHNUKF3EXXZTL6LBP6RJ2CIAAAAAAAAAAAAAIAFRVA65NQ2WBY65WGIG3ZNAYDHDGWFQEPF5BCB26CS7I2Z5UUIAAAAAAAAAAACBYAYDI7XWL6FTSOGLDWXWLICA7A4OUUKJF72STQDPUMZ7EUFZPAGIAAAAAAAAAAAIAAAAAAAAAAABRG44C4NRSFY3TMLRYHI2TIOBTCMAAAAAAAAAAAMJTHAXDMOBOGE4DKLRSGE4DUNJUHAZREAAAAAAAAAAAGEZTQLRWHAXDCOBRFY2TOORVGQ4DGEQAAAAAAAAAAAYTGOBOGY4C4MJYGEXDMMB2GU2DQMYSAAAAAAAAAAADCMZYFY3DQLRRHAYS4OBWHI2TIOBTCIAAAAAAAAAAAMJTHAXDMOBOGE4DCLRYG45DKNBYGMJQAAAAAAAAAABRGM4C4NRYFYYTQMJOGE3DQORVGQ4DGEYAAAAAAAAAAAYTGOBOGY4C4MJYGEXDCNZWHI2TIOBTCMAAAAAAAAAAAMJTHAXDMOBOGE4DCLRRG44TUNJUHAZRGAAAAAAAAAAAGEZTQLRWHAXDCOBRFYYTQMB2GU2DQMYTAAAAAAAAAAADCMZYFY3DQLRRHAYS4MJYGI5DKNBYGMJQAAAAAAAAAABRGM4C4NRYFYYTQMJOGI2DEORVGQ4DGEYAAAAAAAAAAAYTGO

In [ ]:


def getAuthReqForAppdict(appDict,reqFun,ffi=None):
    
    newPermissions,permission_addon=ContainerPermissions(b'_music')
    myAuth,addData=AuthReq(newPermissions,1,1,id=b'newProgId_unique_thing',scope=b'lalali'
                           ,name=b'newProgramName',vendor=b'nobody',app_container=True)    
    
    
    
    infopayload=[]
    if 'permissions' in appDict:
        permissionNum = len(appDict['permissions'])
        permissions = ffi.new(f'ContainerPermissions[{permissionNum}]')
        for idx, onePermission in enumerate(appDict['permissions']):
            permissions[idx].access = PermissionSet(ffi=ffi,**ensure_correct_form(**onePermission['access']))[0]
            #newContainer,addInfo = ContainerPermissions(*ensure_correct_form(ffi,onePermission['name']),access=newPermissionset,ffi=ffi)
            oneContainerName = ffi.new('char[]',ensure_correct_form(ffi,onePermission['name'])[0])
            permissions[idx].cont_name = oneContainerName 
            #permissions[idx]=newContainer[0]
            #infopayload.append(addInfo)
    else:
        permissionNum = 0
        permissions = ffi.NULL
    print(str({item:appDict[item] for item in appDict if not item == 'permissions'}))
    authReq,addData = reqFun(permissions,permissionNum,permissionNum,ffi=ffi,**ensure_correct_form(**{item:appDict[item] for item in appDict if not item == 'permissions'}))
    return authReq,[addData,infopayload]

    

In [5]:
access = {'delete' : True, 'insert' : True, 'manage_permissions' : True, 'read' : True, 'update' : True }
onePermission = { 'access' : access, 'name' : '_testStuff'}

appDict = { 'permissions' : [onePermission], 'id' : 'UniqueName', 'scope' : 'ScopeOfApp', 'name' : 'AppNewName',
          'vendor' : 'AppVendor', 'app_container' : True}

[03:56:39.848] pyS.ffi_in :Thread-5  (     safe_utils.py: 136) [    INFO] action succeeded: < _auth_exe_file_stem_o_cb                    
[03:56:39.850] pyS.ffi_in :Thread-4  (     safe_utils.py: 136) [    INFO] action succeeded: < _auth_set_additional_search_path_o_cb       


In [6]:
myAuth_,addData = safenet.safe_utils.getAuthReqForAppdict(appDict,safenet.safe_utils.AuthReq,myApp.ffi_app)

{'id': 'UniqueName', 'scope': 'ScopeOfApp', 'name': 'AppNewName', 'vendor': 'AppVendor', 'app_container': True}


---

this needs to become nicer

---

In [19]:
grantedAuth = myApp._decode_ipc_msg(msg,myHand)

In [20]:
appItself = myApp._app_registered(myAuth_.app.id,grantedAuth[0],myApp.ffi_app.NULL)

action succeeded


In [21]:
with open('safenet/myNewMutableData_as_bytes','rb') as f:
    readData=f.read()

In [22]:
dataOnSafe=safenet.safeUtils.getffiMutable(readData,myApp.ffi_app)

In [23]:
@myApp.ffi_app.callback("void(void* , FfiResult*, MDataKey*, uint64_t)")
def result_mdata_list_keys(user_data, result, key, size):
    if result.error_code != 0:
        print(myApp.ffi_app.string(result.description))
    if size>0:
        print(myApp.ffi_app.string(key.key))

In [24]:
@myApp.ffi_app.callback("void(void* , FfiResult*, MDataValue*, uint64_t)")
def result_mdata_list_values(user_data, result, value, size):
    if result.error_code != 0:
        print(ffi.string(result.description))
    if size>0:
        print(ffi.string(value.content))

In [25]:
myApp.lib.safe_app.mdata_list_keys(appItself,dataOnSafe,myApp.ffi_app.NULL,result_mdata_list_keys)

b'dask'


In [26]:
myApp.lib.safe_app.mdata_list_values(appItself,dataOnSafe,myApp.ffi_app.NULL,result_mdata_list_values)

b'we get closer...'


---
### why doesn't this work?
---

In [ ]:
myApp.mutableData._mdata_list_keys(appItself,dataOnSafe,myApp.ffi_app.NULL)

In [ ]:
myApp.mutableData._mdata_list_values(appItself,dataOnSafe,myApp.ffi_app.NULL)

In [113]:
access = {'delete' : True, 'insert' : True, 'manage_permissions' : True, 'read' : True, 'update' : True }
containerName ='_music'
onePermission = { 'access' : access, 'name' : containerName}

appDict = { 'permissions' : [onePermission], 'id' : 'UniqueName', 'scope' : 'ScopeOfApp', 'name' : 'AppName',
          'vendor' : 'AppVendor', 'app_container' : True}

In [148]:
myAuth_,addData=safenet.safe_utils.AuthReq(permissions,1,1,id=b'newProgId_unique_thing',scope=b'lalali'
                       ,name=b'newProgramName',vendor=b'nobody',app_container=True,ffi=myApp.ffi_app)

In [117]:
appDict = { 'permissions' : [onePermission], 'id' : 'UniqueName', 'scope' : 'ScopeOfApp', 'name' : 'AppName',
          'vendor' : 'AppVendor', 'app_container' : True}

In [118]:
appDict

{'permissions': [{'access': {'delete': True,
    'insert': False,
    'manage_permissions': True,
    'read': True,
    'update': 13},
   'name': '_music'}],
 'id': 'UniqueName',
 'scope': 'ScopeOfApp',
 'name': 'AppName',
 'vendor': 'AppVendor',
 'ownContainer': True}